In [411]:
import requests
import json
import os

from dotenv import load_dotenv

from datetime import datetime

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [412]:
load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

## Card Data
- Get card information via scryfall.com and delete unnecessary data: 
    - keep only: id, name, mana cost, converted mana cost, type, oracle text, colors, keywords
    - add comments via id
- save as .json

In [413]:
cards = ["Urborg, Tomb of Yawgmoth", "Blood Moon", "Humility", "Opalescence"]

for card in cards:
    response = requests.get(f'https://api.scryfall.com/cards/named?exact={card} ', 
                     headers={'Accept': 'application/json'})
    if response.status_code == 200:
        # Parse the JSON response into a Python dictionary to delete keywords
        data = response.json()
        keywords = ["object", "oracle_id", "mtgo_foil_id", "rarity", "color_identity", "penny_rank", "multiverse_ids", "mtgo_id", "tcgplayer_id", "cardmarket_id", "lang", "released_at", "uri", "scryfall_uri", "layout", "frame_effects", "highres_image", "image_status", "image_uris", "legalities", "games", "reserved", "foil", "nonfoil", "finishes", "oversized", "promo", "reprint", "variation", "set_id", "set", "set_name", "set_type", "set_uri", "rulings_uri", "set_search_uri", "scryfall_set_uri", "prints_search_uri", "collector_number", "digital", "flavor_text", "card_back_id", "artist", "artist_ids", "illustration_id", "border_color", "frame", "security_stamp", "full_art", "textless", "booster", "story_spotlight", "edhrec_rank", "preview", "related_uris", "prices", "tcgplayer_infinite_articles", "tcgplayer_infinite_decks", "edhrec", "purchase_uris", "cardmarket", "cardhoarder"]
        for keyword in keywords:
            if keyword in data:
                del data[keyword]
            
        # convert into json dict
        card = json.loads(json.dumps(data))
        
        # check for additional rulings via id
        card_id = card.get('id')
        if card_id:
            response = requests.get(f'https://api.scryfall.com/cards/{card_id}/rulings', 
                     headers={'Accept': 'application/json'})
        
        # add additional rules to card
            if response.status_code == 200:
                data_rulings = response.json()
                comments = [item.get('comment') for item in data_rulings.get('data', [])]
                card['comments'] = comments
        
        card_name = card.get('name')
        with open(f'./data/cards/{card_name}.json', 'w') as f:
            json.dump(card, f)

## Prompt
- load prompt

In [414]:
with open("prompt_continuouseffects.txt", "r") as file:
    prompt_continuous_effects = file.read()
print(prompt_continuous_effects)

"""
You are judge (aka a referee) in a game of Magic: the Gathering. You will get a {question} regarding {card1} and {card2} and give an answer.
Use the {rules_db} as context for your decision. {card1} and {card2} each contain information of a single Magic: The Gathering card.

The following things are true:
- {card1} and {card2} both have continuous effects.
- The "oracle_text" from {card1} and {card2} describe the effects of the card.
- The "comments" contain rule explanation or examples. Consider them for your decision.
- The "type_line" of the card could be relevant for ruling decision.
- {card1} is played first and has the earlier timestamp, {card2} is played second and has the later timestamp. This may be relevant for the decision, but shouldn't influence your decision which rule to apply.
- Only one rule determine the outcome.

You will follow these steps to make a decision:

1. Give the oracle texts from both cards as well the comments.
2. Determine the layer on which the effec

## RAG
= Retrieved Augmented Generation
- ChatGPT's knowlegde of the rules is from January 2022: for current state the rulebook needs to be retrieved and passed to the LLM
- this data can provide context for the specific questions
### Loader
- simple loader to load .txt data: complete rule book from Magic: The Gathering

In [415]:
#loader = TextLoader("./data/rules_shortened.txt")
loader = TextLoader("./data/rules.txt")
rules_doc = loader.load()

### Splitter
- splitting the text into smaller chunks
- this text splitter splits text recursively by characters: ["\n\n", "\n", " ", ""], so first paragraphs, then sentences, then words
- keeps semantically related pieces together as long as possible

In [416]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
rules_splits = text_splitter.split_documents(rules_doc)
# print(rules_splits[0])

### Vector Store
- for storage of the embeddings
- embedding for capture semantic meaning of the text: that's how context is given for later question
- OpenAI embedding for ChatGPT
- Chroma as open-source embedding database
- finally, construct a retriever out of vectorstore 

In [417]:
rules_db = Chroma.from_documents(documents=rules_splits, embedding=OpenAIEmbeddings())
retriever_rules = rules_db.as_retriever()

- create new LLM: temperature 0 for more consistent, non-creative and therefor comparable output

In [418]:
model = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)

## Case 1: Blood Moon vs. Urborg, Tomb of Yawgmoth
- load card variables from saved data

In [419]:
with open("./data/cards/Urborg, Tomb of Yawgmoth.json", "r") as file:
    card1 = file.read()

with open("./data/cards/Blood Moon.json", "r") as file:
    card2 = file.read()

- load template with prompt and card variables

In [420]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

- Outputparser to parse message response from Chatmodel to str
- check testcases.md to compare expected outcome with ChatGPT's answer

In [421]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the final board state of two cards played with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle texts and comments:
- Urborg, Tomb of Yawgmoth: Each land is a Swamp in addition to its other land types. Comments include that Urborg isn't a Swamp while not on the battlefield, it causes each land on the battlefield to have the land type Swamp, and any land that's a Swamp has the ability "{T}: Add {B}".
- Blood Moon: Nonbasic lands are Mountains. Comments include that nonbasic lands will lose any other land types and abilities they had, they will gain the land type Mountain and the ability "{T}: Add {R}".

2. Determine the layer on which the effects of the cards are applied:
The effects of Urborg, Tomb of Yawgmoth and Blood Moon fall under the dependency layer as they both modify the characteristics of lands.

3. Rule choice:
I choose the Dependency rule (613.8) as it deals with continuous effects that depend on other continuous effects.

4. Summary of the chosen rule:
The Dependency rule (613.8) states that continuous effects apply in timestamp order, but those depending o

- saving the answers as .txt for comparison by date & time: 

In [402]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))

with open(f'./data/answers/case1_{file_name}.txt', 'w') as f:
    f.write(answer)

- cleanup

In [310]:
rules_db.delete_collection()

## Case 2a: Opalescence vs. Humility
- timestamp rule determines the outcome: *Humility* is played last and it's effect overrides *Opalescence*

In [422]:
with open("./data/cards/Opalescence.json", "r") as file:
    card1 = file.read()

with open("./data/cards/Humility.json", "r") as file:
    card2 = file.read()

In [423]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [424]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the final board state of two cards played with continuous effects from Magic: the Gathering?")
print(answer)

1. **Opalescence**:
   - Oracle Text: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value.
   - Comments: The card explains the interaction with **Humility** and provides examples of how the effects apply based on timestamp order.

2. **Humility**:
   - Oracle Text: All creatures lose all abilities and have base power and toughness 1/1.
   - Comments: The card explains the interaction with **Opalescence** and provides examples of how the effects apply based on timestamp order.

3. The effects of both cards are applied in different layers. The effect of **Opalescence** is applied in Layer 4, while the effect of **Humility** is applied in Layer 6.

4. **Rule Choice: Layers (613.1 to 613.7)**
   - This rule governs the order in which continuous effects are applied based on the layers defined in the game.

5. **Judging Decision**:
   - Based on the chosen rule of Layers, the continuous effect of **H

In [425]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

## Case 2b: Humility vs. Opalescence 
- because timestamp is important here, testing the case with changed order

In [426]:
with open("./data/cards/Humility.json", "r") as file:
    card1 = file.read()

with open("./data/cards/Opalescence.json", "r") as file:
    card2 = file.read()

In [427]:
prompt_template = ChatPromptTemplate.from_template(
    template=prompt_continuous_effects,
    partial_variables={"card1": card1, "card2": card2},
)

In [428]:
chain = ({"rules_db": retriever_rules, "question": RunnablePassthrough()} | prompt_template | model | StrOutputParser())
answer = chain.invoke("What is the outcome of two played cards with continuous effects from Magic: the Gathering?")
print(answer)

1. Oracle text and comments:
- Humility: All creatures lose all abilities and have base power and toughness 1/1. Comments include detailed interactions with Opalescence and timestamp order effects.
- Opalescence: Each other non-Aura enchantment is a creature in addition to its other types and has base power and base toughness each equal to its mana value. Comments include interactions with Humility and timestamp order effects.

2. Determine the layer on which the effects of the cards are applied:
The effects of Humility and Opalescence are applied in different layers according to the rules of Magic: The Gathering. Humility's effect of setting all creatures' power and toughness to 1/1 is applied in layer 7b, while Opalescence's effect of turning enchantments into creatures with power and toughness equal to their mana value is applied in layer 4.

3. Rule choice:
The most fitting rule to determine the outcome in this scenario is the Dependency rule (613.8). This rule states that if the e

In [409]:
file_name = str(datetime.now().strftime("%Y-%m-%d %H-%M-%S"))

with open(f'./data/answers/case2_{file_name}.txt', 'w') as f:
    f.write(answer)

- cleanup

In [410]:
rules_db.delete_collection()